# Data Preparation

**Authors:** Abderrahmane Salmi, Ricardo Talarico, Lorenzo Allegrini

In [142]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from utils import drop_columns_if_exists

In [143]:
tracks_df = pd.read_csv("../datasets/tracks_cleaned.csv")
artists_df = pd.read_csv("../datasets/artists_cleaned.csv")

### Check current features list

In [144]:
# Print all feature names
tracks_original_features = tracks_df.columns.tolist()
print(tracks_original_features)

# Print total number of features
print(f"\nTotal number of features: {len(tracks_original_features)}")

['id_artist', 'name_artist', 'title', 'featured_artists', 'language', 'stats_pageviews', 'swear_IT', 'swear_EN', 'swear_IT_words', 'swear_EN_words', 'year', 'month', 'day', 'n_tokens', 'tokens_per_sent', 'char_per_tok', 'lexical_density', 'avg_token_per_clause', 'bpm', 'centroid', 'rolloff', 'flux', 'flatness', 'spectral_complexity', 'pitch', 'loudness', 'album_name', 'album_release_date', 'album_type', 'disc_number', 'track_number', 'duration_ms', 'explicit', 'popularity', 'lyrics']

Total number of features: 35


In [145]:
# Print all feature names
artists_original_features = artists_df.columns.tolist()
print(artists_original_features)

# Print total number of features
print(f"\nTotal number of features: {len(artists_original_features)}")

['id_author', 'name', 'gender', 'birth_date', 'birth_place', 'nationality', 'description', 'active_start', 'active_end', 'province', 'region', 'country', 'latitude', 'longitude']

Total number of features: 14


### Features Created
- swear_ratio: fraction of words in the songs that are swear words
- aggressiveness: measure of the aggressiveness of the song. Computed based on swear_ratio and musical features like bpm, loudness and flux.
- relative_popularity: ranking of the popularity of the song out of all the songs of the artist
- starting_age: the age at which the artist started making music

In [146]:
tracks_df['swear_ratio'] = (tracks_df['swear_IT'] + tracks_df['swear_EN']) / tracks_df['n_tokens']


from sklearn.preprocessing import StandardScaler

features_for_angriness = ['bpm','centroid','flux','rolloff','loudness','swear_ratio']

scaler = StandardScaler()
scaled_features = scaler.fit_transform(tracks_df[features_for_angriness])
scaled_features = pd.DataFrame(scaled_features, columns=features_for_angriness)


tracks_df['aggressiveness'] = (
    (1/6)*scaled_features['centroid']+
    (1/6)*scaled_features['bpm']+
    (1/6)*scaled_features['flux']+
    (1/6)*scaled_features['swear_ratio']+
    (1/6)*scaled_features['loudness']+
    (1/6)*scaled_features['rolloff']
)
# starting age TODO: check if the code works
artists_df['starting_age'] = math.floor((artists_df['active_start'] - artists_df['birth_date']).days / 365)

# relative popularity
tracks_df['relative_popularity'] = (
    tracks_df
    .groupby('name_artist')['popularity']
    .rank(ascending=False, method='dense')
)


TypeError: unsupported operand type(s) for -: 'str' and 'str'

**artist_age**: Older artists may have more experience, different styles, musical inspirations, lyrics richness, etc.

In [147]:
# Ensure datetime parsing for birth dates
# TODO; we already did this in data understand, delete after check
artists_df['birth_date'] = pd.to_datetime(artists_df['birth_date'], errors='coerce')

# Get the birth year
artists_df['birth_year'] = artists_df['birth_date'].dt.year

# Compute artist age
curr_year = datetime.now().year
artists_df['artist_age'] = curr_year - artists_df['birth_year']

# Optional: handle impossible or missing ages (e.g. negative or NaN)
# artists_df.loc[artists_df['artist_age'] < 10, 'artist_age'] = None  # filter unrealistic ages

# Drop temporary columns we created
artists_df = drop_columns_if_exists(artists_df, ['birth_year'])

# Check that the column exists and contains valid values
artists_df[['id_author', 'artist_age']].sample(5)

,id_author,artist_age
82,ART08302616,23.0
12,ART28846313,NaN
13,ART27304446,52.0
56,ART40433104,37.0
29,ART19729064,40.0


**artist_age_at_release**: How old the artist was when they released each song, it allows us to detect age-related patterns in lyrics, energy, popularity, etc. Maybe know if successfull songs are mostly made by younger or older artists, which we can be interpreted as hype vs experience.

In [148]:
# Ensure birth_date is datetime
artists_df['birth_date'] = pd.to_datetime(artists_df['birth_date'], errors='coerce')

# Get birth year directly from birth_date
artists_df['birth_year'] = artists_df['birth_date'].dt.year

# Merge birth year from artists_df into tracks_df
tracks_df = tracks_df.merge(
    artists_df[['id_author', 'birth_year']],
    left_on='id_artist',
    right_on='id_author',
    how='left'
)

# Compute artist age at the time of song release
tracks_df['artist_age_at_release'] = tracks_df['year'] - tracks_df['birth_year']

# Optional: filter unrealistic ages
# tracks_df.loc[(tracks_df['artist_age_at_release'] < 10) | (tracks_df['artist_age_at_release'] > 100), 'artist_age_at_release'] = None

# Drop temporary columns we created
tracks_df = drop_columns_if_exists(tracks_df, ['birth_year'])

# Quick check
tracks_df[['title', 'year', 'name_artist', 'artist_age_at_release']].sample(5)



,title,year,name_artist,artist_age_at_release
9780,IL SENSO DELL’ODIO - Unplugged (Amazon Original),2022.0,Salmo,38.0
2544,Rom Connection,2006.0,Inoki,27.0
10053,Ride or Die,2024.0,Shiva,NaN
10623,Vero hardcore,2011.0,Mondo Marcio,25.0
567,S.O.S. nel mare,2021.0,Nerone,NaN


**release_season**: Season based on release month. Helps us capture trends in release timing (e.g., summer hits).

ps: maybe we convert this to 1-hot encoding

In [149]:
def season_from_month(m):
    if m in [12,1,2]: return 'Winter'
    elif m in [3,4,5]: return 'Spring'
    elif m in [6,7,8]: return 'Summer'
    else: return 'Fall'

tracks_df['release_season'] = tracks_df['month'].apply(season_from_month)

# Quick check
tracks_df[['title', 'month', 'release_season']].sample(5)

,title,month,release_season
1961,Scegli Me,6.0,Summer
7882,Intro (The Best Out Mixtape Vol. 2),5.0,Spring
2603,Ancora Vivo,4.0,Spring
2815,O.L.G.A.,5.0,Spring
3272,Ho Incontrato L’Amore Ma...,12.0,Winter


**has_collaboration**: Binary feature indicating whether a song includes other artists (features guest collaborators).

This feature helps capture the influence of collaborations on a song’s success, because collaborative tracks usually attract fans from multiple artists which makes them perform better.

In [150]:
# A song has a collaboration if 'featured_artists' column is not empty or null
tracks_df['has_collaboration'] = tracks_df['featured_artists'].apply(
    lambda x: 1 if isinstance(x, str) and x.strip() != '' else 0
)

# Quick check
tracks_df[['title', 'featured_artists', 'has_collaboration']].sample(5)

,title,featured_artists,has_collaboration
11100,I Ragazzi Dello Zoo Del Berlin,NaN,0
7890,Money & Egos,NaN,0
7535,Bnb life,NaN,0
5814,Supereroe,NaN,0
5369,Odio volare,Daniele Vit,1


**artist_collab_rate**: It measures an artist’s tendency to collaborate with others.

It shows which artists prefer to work solo VS artists that prefer to collab with others.

In [151]:
# Compute number of collaborative and total songs per artist
artist_collab_stats = (
    tracks_df.groupby('id_artist')['has_collaboration']
    .agg(['sum', 'count'])
    .rename(columns={'sum': 'n_collab_tracks', 'count': 'n_total_tracks'})
)

# Compute collaboration rate (collaborative / total)
artist_collab_stats['artist_collab_rate'] = (
    artist_collab_stats['n_collab_tracks'] / artist_collab_stats['n_total_tracks']
)

# Merge collaboration rate into artists_df
artists_df = artists_df.merge(
    artist_collab_stats[['artist_collab_rate']],
    left_on='id_author',
    right_index=True,
    how='left'
)

# Optional: fill missing values (artists with no tracks)
artists_df['artist_collab_rate'] = artists_df['artist_collab_rate'].fillna(0)

# Quick check
artists_df[['name', 'artist_collab_rate']].sample(5)

,name,artist_collab_rate
84,rose villain,0.338983
85,roshelle,0.136364
97,tony boy,0.201149
3,anna pepe,0.350000
51,johnny marsiglia,0.391304


**song_age**: Older songs might have more pageviews or different audio styles.

In [154]:
curr_year = datetime.now().year
tracks_df['song_age'] = curr_year - tracks_df['year']

# Quick check
tracks_df[['title', 'year', 'song_age']].sample(5)

,title,year,song_age
5877,Champions,2015.0,10.0
2479,La Scatola Nera,2012.0,13.0
8351,Covid-19 Freestyle,2084.0,-59.0
583,Foschi Al Zzoca,2020.0,5.0
5710,Freestyle track,2015.0,10.0


### Check all new features

In [ ]:
# Tracks dataset
# Find the new features
tracks_current_features = tracks_df.columns.tolist()
new_tracks_features = [col for col in tracks_current_features if col not in tracks_original_features]

# Print results
print("New Features Added to artists_df:")
for feature in new_tracks_features:
    print(f"- {feature}")

print(f"\nTotal new features: {len(new_tracks_features)}")

New Features Added to artists_df:
- swear_ratio
- aggressiveness
- id_author
- artist_age_at_release
- release_season
- has_collaboration
- song_age

Total new features: 7


In [ ]:
# Artists dataset
# Find the new features
artists_current_features = artists_df.columns.tolist()
new_artist_features = [col for col in artists_current_features if col not in artists_original_features]

# Print results
print("New Features Added to artists_df:")
for feature in new_artist_features:
    print(f"- {feature}")

print(f"\nTotal new features: {len(new_artist_features)}")

New Features Added to artists_df:
- artist_age
- birth_year
- artist_collab_rate

Total new features: 3
